In [ ]:
import numpy as np
# from scipy.linalg import eig, solve_continuous_are
import control as ct

np.set_printoptions(precision=4, suppress=True)

In [11]:
# Roll dynamics matrices
A_roll = np.array([
    [0, 1, 0],
    [36.5724, 0, 0],
    [-36.5724, 0, 0]
])

B_roll = np.array([
    [0],
    [-0.0142],
    [1.3163]
]) * 10**3

# Pitch dynamics matrices
A_pitch = np.array([
    [0, 1, 0],
    [16.6435, 0, 0],
    [-1.2029, 0, 0]
])

B_pitch = np.array([
    [0],
    [-3.2179],
    [17.6336]
])

# Combine roll and pitch dynamics into full matrices
A_full = np.block([
    [A_roll, np.zeros((3, 3))],
    [np.zeros((3, 3)), A_pitch]
])

B_full = np.block([
    [B_roll, np.zeros((3, 1))],
    [np.zeros((3, 1)), B_pitch]
])

# Display combined matrices
print("Combined A matrix:")
print(A_full)

print("Combined B matrix:")
print(B_full)

Combined A matrix:
[[  0.       1.       0.       0.       0.       0.    ]
 [ 36.5724   0.       0.       0.       0.       0.    ]
 [-36.5724   0.       0.       0.       0.       0.    ]
 [  0.       0.       0.       0.       1.       0.    ]
 [  0.       0.       0.      16.6435   0.       0.    ]
 [  0.       0.       0.      -1.2029   0.       0.    ]]
Combined B matrix:
[[   0.        0.    ]
 [ -14.2       0.    ]
 [1316.3       0.    ]
 [   0.        0.    ]
 [   0.       -3.2179]
 [   0.       17.6336]]


In [13]:
# Penalty weights
# phi_penalty = 1
# phidot_penalty = 1
# thetadot_penalty = 1

# torque_penalty = 1

phi_penalty = 1
phidot_penalty = 1
thetadot_penalty = 1
torque_penalty = 1

# Define the weighting matrices for LQR
Q = np.diag([
    phi_penalty, phidot_penalty, thetadot_penalty,  # roll
    phi_penalty, phidot_penalty, thetadot_penalty  # pitch
])

R = np.diag([
    torque_penalty,
    torque_penalty
])

# Solve the Continuous Algebraic Riccati Equation manually
# P = solve_continuous_are(A_full, B_full, Q, R)

# Compute the LQR gain matrix K manually
# K_scipy = np.linalg.inv(R) @ (B_full.T @ P)

# Compute the LQR gain matrix using the control library
K_control, _, _ = ct.lqr(A_full, B_full, Q, R)

# Display LQR gain matrices
# print("LQR gain matrix K (Scipy):")
# print(K_scipy)

print("LQR gain matrix K (control package):")
print(K_control)

LQR gain matrix K (control package):
[[-1120.2506  -186.2469    -1.        -0.        -0.        -0.    ]
 [    0.         0.         0.       -54.837    -13.549     -1.    ]]


In [14]:
A_cl = A_full - B_full @ K_control

# Compute eigenvalues and eigenvectors of the closed-loop system using control
eigenvalues, eigenvectors = np.linalg.eig(A_cl)

# Display eigenvalues
print("LQR eigenvalues (control package):")
print(eigenvalues)

# Display eigenvectors
evector_1 = eigenvectors[:, 0].real  # First eigenvector
evector_2 = eigenvectors[:, 1].real  # Second eigenvector
evector_3 = eigenvectors[:, 2].real  # Third eigenvector

print("Eigenvector 1 (control package):", evector_1)
print("Eigenvector 2 (control package):", evector_2)
print("Eigenvector 3 (control package):", evector_3)

LQR eigenvalues (control package):
[-1316.3769+0.j        -6.0145+0.032j     -6.0145-0.032j
   -17.9685+0.j        -3.9986+0.3589j    -3.9986-0.3589j]
Eigenvector 1 (control package): [ 0.     -0.0108  0.9999 -0.      0.     -0.    ]
Eigenvector 2 (control package): [ 0.1176 -0.7071 -0.0001 -0.      0.     -0.    ]
Eigenvector 3 (control package): [ 0.1176 -0.7071 -0.0001 -0.      0.     -0.    ]
